<a href="https://colab.research.google.com/github/Matabudi/new/blob/main/VIDScaler_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INSTALL DEPENDENCIES**
Install the required libraries and with GPU support.

In [ ]:
# Install the required libraries and with GPU support.
!pip install tqdm
!pip install ffmpeg-python
!apt update -qq && apt install -y ffmpeg
!pip install --upgrade --force-reinstall --no-deps 'git+https://github.com/kkroening/ffmpeg-python.git'
!pip install ipywidgets
# Ensure CUDA and NVIDIA drivers are installed
!apt-get install -qq cuda-toolkit
from IPython.display import display, HTML, clear_output

# **MOUNT GOOGLE DRIVE**
Run this Cell if you want to mount your Google Drive. Ensure that you define the file paths in each cell if you want to save your files within your drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **DOWNLOAD VIDEO FILE**
Define the 'Video URL' and choose the 'Save Path' before running this cell.

In [ ]:

import requests
from tqdm import tqdm
import os

# Paste a valid URL link of the video you wish to download into the field below.
video_url = ''  # @param {type:"string"}

# Define the save path in Kaggle working directory.
save_path = ''  # @param {type:"string"}

def download_video(url, save_path):
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    block_size = 1024  # 1 Kilobyte
    t = tqdm(total=total_size, unit='iB', unit_scale=True)
    with open(save_path, 'wb') as file:
        for data in response.iter_content(block_size):
            t.update(len(data))
            file.write(data)
    t.close()
    if total_size != 0 and t.n != total_size:
        print("ERROR, something went wrong")
    else:
        print(f"Video downloaded to {save_path}")

# Execute download
download_video(video_url, save_path)

# **VIDEO SIZE COMPRESSION**
Run this cell if you want to downscale the video file size by reducing the resolution.
There are two options for Video Resolution;
###### 1. **480p** - Standard Compression (Standard Quality).
###### 2. **360p** - Maximum Compression (Low Quality).
**(Please note that the amount of compression applied is dependant on the resolution you define in the line of code below.)**

In [ ]:

import ffmpeg
import os
from tqdm import tqdm

# Enter the resolution you want: either "480p" or "360p".
desired_resolution = "480p"  # @param ["480p", "360p"] {type:"string"}

def downscale_video(input_path, resolution="480p"):
    try:
        # Map resolution options to scale values
        resolution_map = {
            "480p": "480",
            "360p": "360"
        }
        if resolution not in resolution_map:
            raise ValueError("Invalid resolution. Choose either '480p' or '360p'.")
        scale_value = resolution_map[resolution]
        # Define the output video path
        output_path = input_path.replace('.mp4', f'_{resolution}.mp4')
        # Probe the video to get file info
        probe = ffmpeg.probe(input_path)
        video_info = next(stream for stream in probe['streams'] if stream['codec_type'] == 'video')
        total_frames = int(video_info['nb_frames'])
        # Function to update the progress bar
        def progress_callback(stats):
            frame = int(stats.get('frame', 0))
            t.update(frame - t.n)
        # Create a progress bar
        with tqdm(total=total_frames, unit='frame') as t:
            # Use h264_nvenc codec for GPU encoding
            process = (
                ffmpeg
                .input(input_path)
                .output(output_path, vf=f"scale=-1:{scale_value}", vcodec="h264_nvenc", preset="fast")
                .global_args('-progress', 'pipe:1', '-nostats')
                .global_args('-y')  # Overwrite output files without asking
                .global_args('-loglevel', 'error')  # Suppress ffmpeg console output
                .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True)
            )
            for line in process.stdout:
                line = line.decode().strip()
                if 'frame=' in line:
                    stats = {}
                    for item in line.split(' '):
                        key, value = item.split('=')
                        stats[key] = value
                    progress_callback(stats)
        print(f"Downscaled video saved to {output_path}")
        # Print original and downscaled file sizes
        original_size = os.path.getsize(input_path) / (1024 * 1024)
        downscaled_size = os.path.getsize(output_path) / (1024 * 1024)
        print(f"Original size: {original_size:.2f} MB")
        print(f"Downscaled size: {downscaled_size:.2f} MB")
    except ffmpeg.Error as e:
        # Print the full error message for debugging
        print(f"ffmpeg error: {e.stderr.decode()}")
    except ValueError as ve:
        # Handle invalid resolution error
        print(ve)

# Set the path of the video file you want to convert
input_video_path = '/content/drive/MyDrive/video.mp4' # @param {type:"string"}
# Execute compression
downscale_video(input_video_path, resolution=desired_resolution)

# **DELETE EXISTING VIDEO FILES**
**CAUTION!** :This will delete any video file within the save path directory that contain the names 'video' & 'downscaled_video'. Once this cell is run, the deletion of the video files cannot be undone.

In [ ]:
# **File Management Operations**

This cell demonstrates basic file management operations in Python, including moving a specified file to a new location and deleting another file.

```python
import os

# Define the file or folder to be moved
source = "/kaggle/working/video.mp4"  # @param {type:"string"}

# Define the location it will be moved to
destination = "/kaggle/output/"  # @param {type:"string"}

# Function to move a file or folder to a new location
def move_file(source, destination):
    try:
        if os.path.exists(source):
            if not os.path.exists(destination):
                os.makedirs(destination)
            new_path = os.path.join(destination, os.path.basename(source))
            os.rename(source, new_path)
            print(f"Moved: {source} -> {new_path}")
        else:
            print(f"Not found: {source}")
    except Exception as e:
        print(f"Error moving {source} to {destination}: {e}")

# Function to delete a specified file
def delete_file(path):
    try:
        if os.path.exists(path):
            os.remove(path)
            print(f"Deleted: {path}")
        else:
            print(f"Not found: {path}")
    except Exception as e:
        print(f"Error deleting {path}: {e}")

# Example usage: Move the file
move_file(source, destination)

# Example usage: Delete a file
delete_file("/kaggle/working/video.mp4")

# **STREAM VIDEO**
Stream the downloaded video within the Kaggle Notebook environment.

In [ ]:
import os
import mimetypes
import ipywidgets as widgets
from IPython.display import display

# Set the path of the video file you want to play.
stream_path = "/content/drive/MyDrive/video.mp4"  # @param {type:"string"}

# List the contents of the directory to ensure the video file exists
print("Directory contents:", os.listdir(os.path.dirname(stream_path)))

# Check if the video file exists
if os.path.exists(stream_path):
    print(f"Video file found at: {stream_path}")
    # Check the file type
    file_type, _ = mimetypes.guess_type(stream_path)
    print(f"File type: {file_type}")
    if file_type == 'video/mp4':
        video = open(stream_path, "rb").read()
        video_widget = widgets.Video(value=video, format='mp4', width=600, height=400)
        display(video_widget)
    else:
        print(f"The file at {stream_path} is not a valid MP4 video file.")
else:
    print(f"Video file not found at: {stream_path}")